# Further exploration of ipyautoui -- should this be fleshed out or dropped???

The easiest way to get a better idea of what ipyautoui can do is to try out its demo. If you have time, feel free to explore!

In [ ]:
from ipyautoui import demo
demo()

# An extension: making the smoothing optional

## More about pydantic and ipyautoui

It might be convenient for the smoothing to be optional, with settings for window size and order available if the user wants to have the smoothed line on their graph.

Our motivation for adding that feature is to illustrate the utility of pydantic and ipyautoui for rapidly refactoring the design of an interface.

## Step 1: make the smoothing settings a separate model

We begin by making a pydnatic model for just the smoothing settings. The fields `window_size` and `polynomial_order` are identical to what we defined in the previous notebook.

In [ ]:
from typing import Annotated

from ipyautoui import AutoUi
from pydantic import BaseModel, Field, model_validator

In [ ]:
class SmoothingSettings(BaseModel, validate_assignment=True):
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

Let's take a quick look at the ipyautoui-generated widget for this to make sure it is what we want.

In [ ]:
ui_smooth = AutoUi(SmoothingSetttings)
ui_smooth

Now we make a class to hold the controls which has two fields: `year_range` and `smoothing`. For `smoothing`, we use the union operator `|` in the type annotation to indicate that the `smoothing` can be either of type `SmoothingSettings` or can be `None`.

In [ ]:
class DataSelector(BaseModel, validate_assignment=True):
    year_range: Annotated[
        tuple[int, int],
        Field(json_schema_extra=dict(autoui='ipywidgets.IntRangeSlider'))
    ] = (1800, 2000)
    smoothing: SmoothingSetttings | None

Next, let's see how ipyautoui renders this.

In [ ]:
ui_data_smooth = AutoUi(DataSelector)
ui_data_smooth

In [ ]:
# Force nullable fields to be shown
ui_data_smooth.show_null = True

In [ ]:
# Force nested models to open
ui_data_smooth.open_nested = True

In [ ]:
...